In [1]:
import numpy as np
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader
from pandas_datareader import data as wb
from pandas import Series, DataFrame

In [2]:
tickers = ['AAPL', 'GOOGL']
data = pd.DataFrame()
for a in tickers:
    data[a] = wb.DataReader(a, data_source = 'iex', start = '2014-05-21')['close']

5y
5y


In [3]:
type(data)

pandas.core.frame.DataFrame

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 2014-05-29 to 2019-05-28
Data columns (total 2 columns):
AAPL     1258 non-null float64
GOOGL    1258 non-null float64
dtypes: float64(2)
memory usage: 29.5+ KB


In [5]:
data.head()

,AAPL,GOOGL
date,,
2014-05-29,83.1915,570.56
2014-05-30,82.8799,571.65
2014-06-02,82.3104,564.34
2014-06-03,83.4743,554.51
2014-06-04,84.4275,553.76


In [6]:
data.tail(2)

,AAPL,GOOGL
date,,
2019-05-24,178.97,1138.61
2019-05-28,178.23,1139.56


In [7]:
returns = np.log(data/data.shift(1))
(returns.head())*100

,AAPL,GOOGL
date,,
2014-05-29,NaN,NaN
2014-05-30,-0.375261,0.190858
2014-06-02,-0.689511,-1.287001
2014-06-03,1.404133,-1.757207
2014-06-04,1.135438,-0.135346


In [8]:
avg = returns.mean()*250
avg

AAPL     0.151537
GOOGL    0.137585
dtype: float64

In [9]:
#Individual Returns
app = returns['AAPL'].mean()*250
z = (app)*100
print(str(round(z,2)) + '%')

15.15%


In [10]:
goog = returns['GOOGL'].mean()*250
x = (goog)*100
print(str(round(x,2)) + '%')

13.76%


In [11]:
#Individual Variances
returns['AAPL'].std()*250**0.5

0.24509581046426532

In [12]:
returns['GOOGL'].std()*250**0.5

0.23143008547278865

In [13]:
#Combined Variances
#Method 1
print(str(round(returns['AAPL'].std()*250**0.5,2))+ '%')
print(str(round(returns['GOOGL'].std()*250**0.5,2)) + '%')

0.25%
0.23%


In [14]:
#Method 2
returns[['AAPL', 'GOOGL']].std()*250**0.5           #Use double brackets to make array two-dimesnional 

AAPL     0.245096
GOOGL    0.231430
dtype: float64

In [15]:
returns[['AAPL', 'GOOGL']].mean()*250

AAPL     0.151537
GOOGL    0.137585
dtype: float64

In [16]:
#Covariance
#If covar = 0 -> Variables are independent
#If covar > 0 -> Variables are moving in the same direction
#If covar < 0 -> Variables are moving in the opposite direction

In [17]:
avar = returns['AAPL'].var()*250
(avar)*100

6.007195630713507

In [18]:
gvar = returns['GOOGL'].var()*250
(gvar)*100

5.355988446194226

In [19]:
cov_matrix = returns.cov()*250
cov_matrix

,AAPL,GOOGL
AAPL,0.060072,0.028847
GOOGL,0.028847,0.053560


In [20]:
corr_matrix = returns.corr()                  #Correlation matrix between RETURNS
corr_matrix                                   #Don't need to annualise correlation matrix

,AAPL,GOOGL
AAPL,1.00000,0.50856
GOOGL,0.50856,1.00000


In [21]:
#Portfolio Variance = [(weight1*return1)*(weight2*return2)^2] 
weights = np.array([0.5,0.5])

In [22]:
pvar = np.dot(weights.T, np.dot(returns.cov()*250, weights))    #'weights.T' = Transposed weights vector (Formula)
f = (pvar) 
f

0.04283137577196705

In [23]:
print(str(round(f,2)) + '%')

0.04%


In [24]:
#Portfolio Volatility
pvol = np.dot(weights.T, np.dot(returns.cov()*250, weights))**0.5     #Addition of '**0.5' to portfolio variance formula
c = (pvol)*100
c

20.69574250225564

In [25]:
print(str(round(c,2)) + '%')

20.7%


In [26]:
#Diversifiable Risk = Portfolio Variance - Weighted Annual Variances
#Weighted Annual Variances = [(weight1^2*variance1) - (weight2^2*variance2)]
#For variances -> Use '.var()*250' instead of '.std()*250**0.5'
weights

array([0.5, 0.5])

In [27]:
n = (avar)
n

0.06007195630713506

In [28]:
m = (gvar)
m

0.053559884461942264

In [29]:
pvar

0.04283137577196705

In [30]:
weights[0]**2

0.25

In [31]:
weights[1]**2

0.25

In [32]:
dr = pvar - (0.25*avar) - (0.25*gvar)      #From portfolio variance formula
(dr)*100

1.4423415579697718

In [33]:
print(str(round(dr,3)*100) + '%')

1.4000000000000001%


In [34]:
#Non-Diversifiable Risk = Portfolio Variance - Diversifiable Risk
#Method 1
ndr1 = pvar - dr
(ndr1)*100

2.840796019226933

In [35]:
#Method 2
ndr2 = (weights[0]**2*avar) + (weights[1]**2*gvar)
(ndr2)*100

2.840796019226933

In [36]:
ndr1 == ndr2                  #Verification (Optional)

True